In [1]:
# para buscar archivos
from glob import glob
# super excel para analizar
import pandas as pd
# para leer los datos .json
import json
# para hacer busquedas en strings
import re
# para operaciones numericas
import numpy as np
# para que quede mas lindo
import warnings
warnings.filterwarnings('ignore')

In [314]:
# Para bajar los datos (Mati)
# aws s3 sync s3://unket/gyani ./data

In [2]:
def data2df(f):
    """recibe path al archivo y devuelve DataFrame"""
    with open(f, 'rb') as inp:
        j = inp.read()
        #j = re.sub(r"\\+", r"\\", j)
        data = json.loads(j)
        d = json.loads(data)
        df = pd.read_json(d)
    return df #pd.read_json(j)

In [3]:
files = glob('./datos/*.json')

In [4]:
def wrangle(row):
    etapas=['welcome', 'consent', 'demography1', 'demography2', 'if_otroo',
            'theoretical_orientation_ticks', 'if_tcc', 'if_psan', 'if_hum', 'if_ecl',
            'if_etc', 'workplace_ticks', 'if_otro', 'influence', 'sources',
            'research_attitudes_1', 'research_attitudes_2', 'comments']
    # nombramos la etapa usando el internal id
    etapa = etapas[int(re.search('-(\d+)', row['internal_node_id']).group(1))]
    # leemos las responses y formateamos el dict 
    try:
        responses = json.loads(row['responses'])
        return {f"{etapa}-"+ re.search('(\d+)', k).group(0) if k != 'undefined' else etapa: v for k,v in responses.items()}
    except:
        return {etapa:row['responses']}

In [5]:
def get_sujeto(df):
    # aplicamos la funcion wrangle y armamos una fila con toda la info
    return pd.DataFrame.from_dict({k:v for l in df.apply(wrangle, axis=1) for k,v in l.items()}, orient='index').T

In [6]:
f = files[10]

In [7]:
df = pd.DataFrame()
for f in files:
    df_ = data2df(f)
    df=df.append(get_sujeto(df_))

In [ ]:
columns  = ['', 'welcome', 'consent','demography1','gender', 
            'educcation', 
            'theoretical_orientation',
           'if_theoretical_orientation',
           'workplace',
           'if_workplace',
            'evidencia',
            'pers_experiencia', 
            'entrenamiento',
             'pref_consult', 
            'intuicion',
            'docentes', 
            'pers_terapia',
            'exp_pers_consult', 'inv_control', 'supervision', 'inv_casos',
           'pares','libros', 'obs_casos', 'medidas_resultados', 'guias_clinicas',
            
           ]

In [ ]:
{'demography2-0': 'gender', 'demography2-1':'education', }

,comments,consent,demography1,demography2-0,demography2-1,if_ecl,if_etc,if_hum-0,if_otro,if_psan-0,...,sources-2,sources-3,sources-4,sources-5,sources-6,sources-7,sources-8,theoretical_orientation_ticks-0,welcome,workplace_ticks-0
0,,NaN,,Femenino,Licenciatura de grado,,NaN,NaN,,NaN,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Ecléctica (más de una/de todo un poco/depende ...,NaN,"[Obra Social, Otro]"
0,,NaN,20,Masculino,Doctorado,NaN,NaN,NaN,NaN,NaN,...,A veces,A veces,A veces,A veces,Nunca,A veces,Siempre,Terapias Cognitivas/Comportamentales,NaN,[Privado]
0,,NaN,,Femenino,Licenciatura de grado,sarasa,NaN,NaN,NaN,NaN,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Ecléctica (más de una/de todo un poco/depende ...,NaN,[Obra Social]
0,1. No me obligó a elegir una opción en el subg...,NaN,f,Masculino,Carrera de especialización,NaN,NaN,NaN,f,NaN,...,Siempre,Siempre,Siempre,Siempre,Siempre,Siempre,Siempre,Sistémica,NaN,[Otro]
0,:),NaN,6,Masculino,Licenciatura de grado,NaN,NaN,NaN,hospital,NaN,...,Siempre,A veces,Siempre,Nunca,Siempre,A veces,Siempre,Terapias Cognitivas/Comportamentales,NaN,[Otro]
0,,NaN,,Femenino,Licenciatura de grado,NaN,NaN,NaN,NaN,NaN,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Transpersonal,NaN,[Público]
0,Muy interesante,NaN,12,Masculino,Doctorado,NaN,NaN,NaN,NaN,NaN,...,Siempre,Siempre,Siempre,A veces,A veces,Siempre,Siempre,Terapias Cognitivas/Comportamentales,NaN,[Privado]
0,a,NaN,1,Masculino,Master,NaN,as,NaN,NaN,NaN,...,A veces,A veces,A veces,A veces,A veces,A veces,A veces,Otra,NaN,[Obra Social]
0,S,NaN,Ñj,No deseo contestar,Doctorado,NaN,NaN,NaN,NaN,[Psicología analítica (junguiana)],...,A veces,A veces,A veces,A veces,A veces,A veces,A veces,Psicoanálisis,NaN,[Obra Social]
0,Funciona perfecto,NaN,5,Masculino,Licenciatura de grado,NaN,NaN,NaN,NaN,NaN,...,A veces,Siempre,Siempre,A veces,A veces,A veces,A veces,Terapias Cognitivas/Comportamentales,NaN,[Público]


In [ ]:
{'demography2-0': ''}

In [11]:
df.to_excel('preliminary_results.xlsx')

---

In [8]:
def concat_rows(df):
    rows = df.iloc[0,:]
    for i in range(len(df)-1):
        rows = pd.concat([rows, df.iloc[i+1,:]])
    return rows

In [49]:
concatenated = concat_rows(response)

In [48]:
pages = ['ins','demograficos', 'demografico 2', 'orientacion','sector', 'eleccion_orientacion']
demograficos = ['edad', 'exp_clinica', 'horasXsemana']
demograficos2 = ['genero', 'Nivel_educativo']
orientacion = ['orientacion']
sector = ['sector']
eleccion_orientacion = ['evidencia','pers_experiencia', 'entrenamiento',
                        'pref_consult', 'intuicion', 'docentes', 'pers_terapia']
mejorar = ['exp_pers_consult', 'inv_control', 'supervision', 'inv_casos',
           'pares','libros', 'obs_casos', 'medidas_resultados', 'guias_clinicas' ]
opinion1 = ['dispuesto_cambio', 'cons_mas_compejos', 'estar_al_dia', 
 'cons_otros_trat', 'formacion_inv', 'ter_no_manual', 'superv_evidencia',
 'diag_simples', 'tbe_eficientes', 'trat_no_probados', 'trat_individual']
opinion2 = ['incorporar_ciencia', 'tiempo_tbe', 'intentar_dif', 
            'capacit_costosa', 'interes_aorender', 'no_se_tbe', 'uso_tbe', 'entr_no_tbe',
           'alianza', 'ter_iguales', 'empleador_sin_fondos', 'exp_mejor_evi']
coments = ['preg_imprecisa', 'otras_pregs', 'otro']

cols = demograficos + (12-len(demograficos))*['NaN'] + demograficos2 + (12-len(demograficos2))*['NaN'] + orientacion+ (12-len(orientacion))*['NaN']+ ['otro_orient']+ 11*['NaN']+ sector+(12-len(sector))*['NaN'] + ['otro_sector']+ 11*['NaN']+eleccion_orientacion + (12-len(eleccion_orientacion))*['NaN']+ mejorar +(12-len(mejorar))*['NaN']+ opinion1 + (12-len(opinion1))*['NaN']+opinion2 +(12-len(opinion2))*['NaN']+ coments +(12-len(coments))*['NaN']
len(cols)

132